In [1]:
import os
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import random
import string
from multiprocessing import Pool

# Define input parameters
tmpdir = '.'                    # Parent Directory to write/read data
size   = 1000000                # Length of global dataframe
npart  = 8                      # Number of dataframe partitions
nprocs = 8                      # Number of processes to use
path = tmpdir + '/test_dataset' # Parquet dataset directory

### Investigating parallel I/O with Pandas, Apache Arrow and Parquet

- Rick Zamora (original: 5/14/2019)

**Update 6/11/2019**: This document has been modified to reflect recent progress in metadata handling within `arrow.parquet` (e.g. [PR#4405](https://github.com/apache/arrow/pull/4405)).


This notebook includes a simple exploration of the pyarrow-parquet writer/reader interface. The general goal is to discover the requirements for efficient parallel processing of partitioned pandas datasets.


To begin this exploration, we generate a *large* global pandas dataframe, and partition it into `npart` pieces.

In [2]:
# Define the global dataframe
df = pd.DataFrame(
    {
    'a': [random.choice(['A', 'B', 'C', 'D']) for c in range(size)], 
    'b': np.arange(size),
    'c': [random.random() for c in range(size)], 
    }
)

# Break the global dataframe into partitions
lsize = size//npart
df_part = [ df.iloc[ x * (size//npart) : size if (x == npart-1) else (x * (size//npart) + size//npart) ] for x in range(npart) ]
df_part[npart-1].head()

,a,b,c
875000,B,875000,0.986055
875001,B,875001,0.633334
875002,B,875002,0.024982
875003,B,875003,0.609053
875004,A,875004,0.675325


Above, we are showing the `head()` of the last partition. In order to write each of these partions, we define the `write_partition` function, which returns a *metadata* object for each partition. Note that the metadata object is actually a dictionary, with `'meta'`, and `'schema'` keys.

In order to write each *partition* of the dataframe, we define a specialized `write_partition` function that takes in the partition (a dataframe), as well as the file name (file-system path). We will use the `write_table` function from `arrow.parquet` to perform the actual I/O opperation.  This funtion allows the user to pass in an empty list object using the `metadata_collector` keyword argument.  In the cell below, `metadata_list` will be populated with a parquet-metadata (`pyarrow._parquet.FileMetaData`) object for each dataset partition. After collecting this metadata object, we need to use the `set_file_path` method to correctly define the location of the **data** corresponding to this partition.

In [3]:
# Define `write_partition` funciton
def write_partition(df, i, ipath):
    filepath = ipath + "/part.%i.parquet" % i
    t = pa.Table.from_pandas(df)
    metadata_list = []
    with open(filepath, "wb") as fil:
        pq.write_table(
            t, 
            fil, 
            metadata_collector=metadata_list
        )
        # Set file path attribute in metadata:
        metadata_list[0].set_file_path("part.%i.parquet" % i)
    # Return metadata & schema
    return {'meta': metadata_list[0], 'schema': t.schema}

In order to write the pandas dataframe in parallel, we can use a simple `multiprocessing.Pool` approach.

In [4]:
# Write the datafram in parallel (to multiple parquet files)
def fwrite(args):
    i = args[0]
    part = args[1]
    ipath = args[2]
    filename = ipath + "/part.%i.parquet" % i
    meta = write_partition(part, i, ipath)
    return meta

fargs = [ [i, part, path] for i, part in enumerate(df_part) ]
os.system('rm -rf ' + path)
os.system('mkdir ' + path)
p = Pool(nprocs)
%time meta = p.map(fwrite, fargs)

# Check that there are multiple files
files = os.listdir(path)
files

CPU times: user 48 ms, sys: 12 ms, total: 60 ms
Wall time: 128 ms


['part.2.parquet',
 'part.6.parquet',
 'part.7.parquet',
 'part.4.parquet',
 'part.5.parquet',
 'part.3.parquet',
 'part.0.parquet',
 'part.1.parquet']

Here, we can see that each of the `npart` partitions is written as a separate file called `'part.<part>.parquet'` (where `part` is just an integer in this case). After collecting the file metadata (and schemas) from all partitions within the `meta` list, we can check that the `file_path` attribute is properly set for one of the partitions...

In [5]:
meta[0]['meta'].to_dict()

{'created_by': 'parquet-cpp version 1.5.1-SNAPSHOT',
 'num_columns': 3,
 'num_rows': 125000,
 'num_row_groups': 1,
 'row_groups': [{'num_columns': 3,
   'num_rows': 125000,
   'total_byte_size': 2063898,
   'columns': [{'file_offset': 31587,
     'file_path': 'part.0.parquet',
     'physical_type': 'BYTE_ARRAY',
     'num_values': 125000,
     'path_in_schema': 'a',
     'is_stats_set': True,
     'statistics': {'has_min_max': True,
      'min': b'A',
      'max': b'D',
      'null_count': 0,
      'distinct_count': 0,
      'num_values': 125000,
      'physical_type': 'BYTE_ARRAY'},
     'compression': 'SNAPPY',
     'encodings': ('PLAIN_DICTIONARY', 'PLAIN', 'RLE'),
     'has_dictionary_page': True,
     'dictionary_page_offset': 4,
     'data_page_offset': 40,
     'total_compressed_size': 31583,
     'total_uncompressed_size': 31575},
    {'file_offset': 797909,
     'file_path': 'part.0.parquet',
     'physical_type': 'INT64',
     'num_values': 125000,
     'path_in_schema': 'b',

Since we are collecting the metadata/schema information for each of the partitions that is written, we can write the metadata and schema information to specialized files called `_metadata` and `_common_metadata`, respectively.

First, we can write `_common_metadata` using the `write_metadata()` function available in `pyarrow.parquet`.

In [6]:
# Write a `_common_metadata` (schema) file using the 0th partition
# (Note that this is not actually needed...)
schema = meta[0]['schema']
common_metadata_path = path + '/_common_metadata' 
with open(common_metadata_path, "wb") as f:
    pq.write_metadata(schema, f)
schema

a: string
b: int64
c: double
metadata
--------
{b'pandas': b'{"index_columns": [{"kind": "range", "name": null, "start": 0, "'
            b'stop": 125000, "step": 1}], "column_indexes": [{"name": null, "f'
            b'ield_name": null, "pandas_type": "unicode", "numpy_type": "objec'
            b't", "metadata": {"encoding": "UTF-8"}}], "columns": [{"name": "a'
            b'", "field_name": "a", "pandas_type": "unicode", "numpy_type": "o'
            b'bject", "metadata": null}, {"name": "b", "field_name": "b", "pan'
            b'das_type": "int64", "numpy_type": "int64", "metadata": null}, {"'
            b'name": "c", "field_name": "c", "pandas_type": "float64", "numpy_'
            b'type": "float64", "metadata": null}], "creator": {"library": "py'
            b'arrow", "version": "0.13.1.dev281+g5999609"}, "pandas_version": '
            b'"0.24.2"}'}

Notice that the `pq.write_metadata()` call above uses the `schema` object for the 0th partition. This actually means that not **all** information in `_common_metadata` will be true for the entire dataset. More specifically, the `"index_columns"` item will have something like this:

```[{"kind": "range", "name": null, "start": 0, "stop": 125000, "step": 1}]```
            
Therefore, the `"start"` and `"stop"` values will correspond to the 0th partition, rather than the global dataset.

Although we just took the time to write out a `_common_metadata` file, it does not currently make much sense to use the file to read back the dataset.  I say this, because that file does not seem include the type of metadata we really need for typical data-processing tasks (such as the number of partitions, the schema, and the row statistics).

Instead, what we really want is the so-called `_metadata` file, which (by convention) contains the aggregated metadata for all partitions within a single file (including row-group statistics). To write this file, we must aggregate our list of metadata objects into a single metadata object (using `append_row_groups`). Then we use `` to write a dedicated metadata file...

In [15]:
_meta = meta[0]['meta']
for i in range(1,len(meta)):
    _meta.append_row_groups(meta[i]['meta'])
metadata_path = path + '/_metadata'
with open(metadata_path, "wb") as f:
    _meta.write_metadata_file(f)

Now that we have an aggregated metadata file, we can read that file (and only that file) to learn a lot about the dataset without using a file-path glob and parsing the metadata of every single file in the dataset. Since `_metadata` specifies the file-system path for each row-group column in the dataset, the files can also have a complex/nested/remote organization.

In [19]:
# Read back the metadata
meta = pq.read_metadata(metadata_path)
md = meta.to_dict()

print("Number of dataset rows: "+str(md['num_rows']))
print("Number of dataset columns: "+str(md['num_columns']))
print("Number of dataset row_groups: "+str(md['num_row_groups']))

Number of dataset rows: 3125000
Number of dataset columns: 3
Number of dataset row_groups: 25


As an example/demonstration, we can use `multiprocessing.Pool` again to generate a *metadata* dataframe (note: there are probably much more efficient ways to parse the `_metadata` file)...

In [50]:
# Get files and global statistics
def fparse(args):
    i = args[0]
    rg = args[1]
    rg_dict = {'rgID': [], 'colID': [], 'cmin': [], 'cmax': [], 'file_path': []}

    for j, col in enumerate(rg['columns']):
        rg_dict['rgID'].append(i)
        rg_dict['colID'].append(col['path_in_schema'])
        if isinstance(col['statistics']['min'], bytes):
            rg_dict['cmin'].append(col['statistics']['min'].decode("utf-8"))
            rg_dict['cmax'].append(col['statistics']['max'].decode("utf-8"))
        else:
            rg_dict['cmin'].append(col['statistics']['min'])
            rg_dict['cmax'].append(col['statistics']['max'])
        rg_dict['file_path'].append(col['file_path'])
    return pd.DataFrame(rg_dict)

fargs = [ [i, rg] for i, rg in enumerate(md['row_groups']) ]
p = Pool(nprocs)
%time frames = p.map(fparse, fargs)

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 16.7 ms


In [67]:
meta_df = pd.concat(frames)
meta_df.head()

,rgID,colID,cmin,cmax,file_path
0,0,a,A,D,part.0.parquet
1,0,b,0,124999,part.0.parquet
2,0,c,4.08375e-06,0.999991,part.0.parquet
0,1,a,A,D,part.1.parquet
1,1,b,125000,249999,part.1.parquet


On systems where each file access is slow, it can be very useful to work with an in-memory dataframe (like this one) to avoid openeing files when it isn't necessary. For example, if you are just looking for the maximum value in column `'c'`, you can use the following approach to determine the file that needs to be parsed (in this case, `part.5.parquet`):

In [68]:
# Find maximum value for column == col_id
col_id = 'c'
c_max = meta_df.groupby(['colID']).max()['cmax'][col_id]
mdf = meta_df[ meta_df['cmax'] == c_max ]
mdf[ mdf['colID'] == col_id ]

,rgID,colID,cmin,cmax,file_path
2,8,c,6.20876e-06,0.999999,part.5.parquet
2,15,c,6.20876e-06,0.999999,part.5.parquet
2,22,c,6.20876e-06,0.999999,part.5.parquet


We can also use the `_metadata` file to get a list of all files containing the partitions for out dataset.  This is useful when the file-system glob operations are slow, but is also critical when the data-files are not organized in a simple/predictible way.

In [71]:
%time files = list(set(meta_df['file_path']))
files

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 93.2 µs


['part.3.parquet',
 'part.0.parquet',
 'part.6.parquet',
 'part.4.parquet',
 'part.7.parquet',
 'part.5.parquet',
 'part.2.parquet',
 'part.1.parquet']

Using this list of metadata files, we can do the reverse of the parallel *write* operation above.

In [41]:
%time dataset = pq.ParquetDataset(path)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 6.49 ms


In [42]:
pieces = dataset.pieces
piece = pieces[0]
%time md = piece.get_metadata(lambda fn: pq.ParquetFile(open(fn, mode="rb")))
columns = md.schema.names
md

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 716 µs


  created_by: parquet-cpp version 1.5.1-SNAPSHOT
  num_columns: 3
  num_rows: 125000
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 991

As shown above, rather than reading the metadata from a dedicated metadata file, we can use the `dataset` obect to read the metadata from each piece (partition).  If we are interested in the row statistics of each partition, we can iterate trhough each file and generate our own `stats` structure. For example:

In [43]:
row_groups = [
    piece.get_metadata(
        lambda fn: pq.ParquetFile(open(fn, mode="rb"))
    ).row_group(0)
    for piece in pieces
]
stats = []
for row_group in row_groups:
    s = {"num-rows": row_group.num_rows, "columns": []}
    for i, name in enumerate(columns):
        column = row_group.column(i)
        d = {"name": name}
        if column.statistics:
            d.update(
                {
                    "min": column.statistics.min,
                    "max": column.statistics.max,
                    "null_count": column.statistics.null_count,
                }
            )
        s["columns"].append(d)
    stats.append(s)
stats[0]

{'num-rows': 125000,
 'columns': [{'name': 'a', 'min': b'A', 'max': b'D', 'null_count': 0},
  {'name': 'b', 'min': 0, 'max': 124999, 'null_count': 0},
  {'name': 'c', 'min': 6.62184e-06, 'max': 0.999997, 'null_count': 0}]}

Using the `pieces` member of `ParquetDataset` class, it is straightforward to read back each partition of the dataset in parallel. Since we know the names of the columns and the *pieces* to iterate through, we can use the same `multiprocessing.Pool` approach that we used to write the dataset:

In [44]:
def read_partition(piece, columns):
    with open(piece.path, mode="rb") as f:
        table = piece.read(
            columns=columns,
            use_pandas_metadata=True,
            file=f,
            use_threads=False
        )
    df = table.to_pandas(use_threads=False)
    return df[list(columns)]

In [45]:
def fread(args):
    piece = args[0]
    columns = args[1]
    dfrd = read_partition(piece, columns=columns)
    return dfrd

fargs = [ [piece, columns] for piece in pieces ]
p = Pool(nprocs)
%time df_part_rd = p.map(fread, fargs)

CPU times: user 24 ms, sys: 20 ms, total: 44 ms
Wall time: 344 ms


Here, we can also confirm that each partition of `df_part_rd` is the same as `df_part`:

In [46]:
assert(df_part_rd[i] == df_part[i] for i in range(npart))

Warning: This document should be viewed as my own personal notes.  There may be serious mistakes/errors.